In [9]:
# VERİ SETİNİ GETİRME
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

data = sns.load_dataset("tips")
df = pd.DataFrame(data)

# Eksik Verileri Ortalama ile Doldurma
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(df[["total_bill"]])
df[["total_bill"]] = imputer.transform(df[["total_bill"]])

# Kategorik Olmayan Değişkenler
total_bill = df[["total_bill"]]
tip = df[["tip"]]
size = df[["size"]]

# Ktegorik Değişkenler İçin 0-1 Dönüşümü
smoker = pd.get_dummies(df[["smoker"]])
time = pd.get_dummies(df[["day"]])

# Kukla Değişkeni Silme
smoker = smoker.drop("smoker_Yes", axis=1)

# Verileri Birleştirme
new_data = pd.concat([total_bill, size, smoker, time, tip], axis=1)

# Train ve Test Belirleme
training = new_data.iloc[:, 0:7]
testing = new_data.iloc[:, 7:8]

# Train Test Ayırma
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(training, testing, test_size=0.30, random_state=42)

### Model

In [10]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(x_train, y_train)

LinearRegression()

In [11]:
from sklearn.preprocessing import PolynomialFeatures

polynomial_reg = PolynomialFeatures(degree=3)
xp = polynomial_reg.fit_transform(x_train)

In [12]:
linear_reg2=LinearRegression()
linear_reg2.fit(xp, y_train)

LinearRegression()

### İstenilen Tahmin

In [13]:
print(linear_reg2.predict(polynomial_reg.fit_transform([[16.99,2,1,0,0,0,1]])))

[[2.82949043]]


### MSE

In [15]:
from sklearn.metrics import mean_squared_error

y_pred = linear_reg2.predict(polynomial_reg.fit_transform(x_test))

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE: ", rmse)

Test RMSE:  1.4406716038278844


### Tahmin v Gerçek

In [16]:
y_pred = linear_reg2.predict(polynomial_reg.fit_transform(x_test))

y_tahmin=pd.DataFrame(data = y_pred)
y_normal = pd.DataFrame(data = y_test)
y_tahmin.index = y_normal.index

result=pd.concat([y_tahmin, y_normal],axis=1)
result.columns=[["Tahmin","Gerçek"]]

print(result)

       Tahmin Gerçek
24   3.245409   3.18
6    1.277304   2.00
153  4.339456   2.00
211  2.968969   5.16
198  2.385430   2.00
..        ...    ...
165  3.169453   3.48
154  3.813692   2.00
216 -1.873190   3.00
79   2.882576   2.71
29   3.240724   3.00

[74 rows x 2 columns]
